In [1]:
import argparse
import os
import pickle

import torch as T
import yaml
from torch.optim import Adam

from data_pipeline import DatasetsGenerator
from evaluation import Evaluate
from model import Model
from training import set_model_to_train_novel, train_loop

def load_settings(settings_path: str):
    with open(settings_path, 'r') as f:
        return yaml.safe_load(f)

In [2]:
settings = 'settings/model_testing.yaml'

conf = load_settings(settings)

if os.path.exists(conf['training']['save_base_weights_dir']) and conf["train_base"]:
    raise ValueError("Cannot overwrite weights")

debug_mode = conf['debug']['debug_mode_active']
device = conf['device']

K = conf['data']['K']
val_K = conf['data']['val_K']
test_K = conf['data']['test_K']
n_repeats_novel_train = conf['training']['repeat_novel_training']

if isinstance(K, int): K = [K]
if isinstance(val_K, int): val_K = [val_K]
if isinstance(test_K, int): test_K = [test_K]

In [3]:
# Dataset generator. Only one of these has to be instantiated. It always returns
dataset_gen = DatasetsGenerator(
    annotations_path = conf['paths']['annotations_path'],
    images_dir = conf['paths']['images_dir'],
    novel_class_ids_path = conf['paths']['novel_classes_ids_path'],
    train_set_path = conf['paths']['train_base_annotations_path'],
    val_set_path = conf['paths']['val_base_annotations_path'],
    test_set_path = conf['paths']['test_base_annotations_path'],
    use_fixed_novel_sets = conf['data']['use_fixed_sets'],
    novel_train_set_path = conf['paths']['train_novel_annotations_path'],
    novel_val_set_path = conf['paths']['val_novel_annotations_path'],
    novel_test_set_path = conf['paths']['test_novel_annotations_path']
)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=1.62s)
creating index...
index created!


In [4]:
base_train, base_val, base_test = dataset_gen.get_base_sets()

In [5]:
len(base_val)

2013

In [6]:
base_val.transform

In [22]:
x = 5
for i, k in enumerate(base_val):
    sample, trans_landmarks, original = k
    img = sample[0]
    regress_label, heatmap_base, heatmap_novel = sample[1]
    print(sample[0].shape)
    print(regress_label.shape)
    print(heatmap_base.shape)
    print(heatmap_novel.shape)
    print(trans_landmarks)
    print()

    if i == x:
        break

torch.Size([3, 512, 512])
torch.Size([4, 128, 128])
torch.Size([240, 128, 128])
torch.Size([0, 128, 128])
[{'center_point': (281.5226586102719, 257.58024691358025), 'size': (156.2296072507553, 418.7654320987654), 'category_id': 26}]

torch.Size([3, 512, 512])
torch.Size([4, 128, 128])
torch.Size([240, 128, 128])
torch.Size([0, 128, 128])
[{'center_point': (205.24521739130435, 247.70626349892007), 'size': (326.7895652173913, 484.35421166306696), 'category_id': 26}]

torch.Size([3, 512, 512])
torch.Size([4, 128, 128])
torch.Size([240, 128, 128])
torch.Size([0, 128, 128])
[{'center_point': (165.67375886524823, 268.97465886939574), 'size': (442.09929078014187, 438.14424951267057), 'category_id': 26}]

torch.Size([3, 512, 512])
torch.Size([4, 128, 128])
torch.Size([240, 128, 128])
torch.Size([0, 128, 128])
[{'center_point': (278.8973607038123, 138.0523560209424), 'size': (246.99120234604106, 299.3368237347295), 'category_id': 26}]

torch.Size([3, 512, 512])
torch.Size([4, 128, 128])
torch.S

In [28]:
list(base_val.coco.cats).index(26)

40

In [32]:
heatmap_base[40, 65:74, 65:74]

tensor([[0.9753, 0.9821, 0.9871, 0.9900, 0.9910, 0.9900, 0.9871, 0.9821, 0.9753],
        [0.9802, 0.9871, 0.9920, 0.9950, 0.9960, 0.9950, 0.9920, 0.9871, 0.9802],
        [0.9831, 0.9900, 0.9950, 0.9980, 0.9990, 0.9980, 0.9950, 0.9900, 0.9831],
        [0.9841, 0.9910, 0.9960, 0.9990, 1.0000, 0.9990, 0.9960, 0.9910, 0.9841],
        [0.9831, 0.9900, 0.9950, 0.9980, 0.9990, 0.9980, 0.9950, 0.9900, 0.9831],
        [0.9802, 0.9871, 0.9920, 0.9950, 0.9960, 0.9950, 0.9920, 0.9871, 0.9802],
        [0.9753, 0.9821, 0.9871, 0.9900, 0.9910, 0.9900, 0.9871, 0.9821, 0.9753],
        [0.9684, 0.9753, 0.9802, 0.9831, 0.9841, 0.9831, 0.9802, 0.9753, 0.9684],
        [0.9597, 0.9665, 0.9713, 0.9743, 0.9753, 0.9743, 0.9713, 0.9665, 0.9597]])

In [39]:
regress_label[:, 65:74, 65:74]

tensor([[[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000, 155.8883,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000]],


In [ ]:
for i, k in enumerate(base_test):
    sample, trans_landmarks, original = k
    print(sample.shape)
    print(trans_landmarks)
    break